In [1]:
!pip install yfinance


In [157]:
import sklearn
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, auc, recall_score

In [27]:
xmr = yf.Ticker("XMR-USD")
df_xmr_train = yf.download(tickers = "XMR-USD",
                     start = '2017-11-09',
                     end = '2023-09-09',
                     interval = "1d")
df_xmr_test = yf.download(tickers = "XMR-USD",
                     start = '2023-09-08',
                     end = '2023-10-07',
                     interval = "1d")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [185]:
df_train = pd.read_csv('/content/train.csv')
df_test = pd.read_csv('/content/test.csv')
df_train.columns

Index(['Unnamed: 0', 'Date', 'XMR-USD Open', 'XMR-USD High', 'XMR-USD Low',
       'XMR-USD Close', 'XMR-USD Volume', 'Change', 'Percentage Change',
       'Increase', 'Target', 'RSI', 'SMA', 'EMA', 'GC=F Open', 'GC=F High',
       'GC=F Low', 'GC=F Close', 'GC=F Volume', 'GoldChangePercent',
       '^GSPC Open', '^GSPC High', '^GSPC Low', '^GSPC Close', '^GSPC Volume',
       'GSPCChangePercent', 'HG=F Open', 'HG=F High', 'HG=F Low', 'HG=F Close',
       'HG=F Volume', 'SilverChangePercent', 'ETH-USD Open', 'ETH-USD High',
       'ETH-USD Low', 'ETH-USD Close', 'ETH-USD Volume', 'EthChangePercent',
       'BTC-USD Open', 'BTC-USD High', 'BTC-USD Low', 'BTC-USD Close',
       'BTC-USD Volume', 'BtcChanePercent'],
      dtype='object')

In [204]:
def cvscore(X, n_hist=1):
  X_test = df_test[X.columns].copy()
  Y_test = df_test['Target'].copy()
  Y = df_train['Target'].copy()

  price = df_train['XMR-USD Close']
  for i in range(n_hist):
    X[f'XMR{-i}'] = np.roll(price, i)

  X = X[n_hist:]
  Y = Y[n_hist:]

  na = X[X.isna().any(axis=1)].index
  X=X.drop(na)
  Y=Y.drop(na)
  print(X[X.isna().any(axis=1)].index)

  clf = tree.DecisionTreeClassifier(random_state=42)
  cvs = cross_val_score(clf, X, Y, cv=10)

  price = df_test['XMR-USD Close']
  for i in range(n_hist):
    X_test[f'XMR{-i}'] = np.roll(price, i)

  X_test = X_test[n_hist:]
  Y_test = Y_test[n_hist:]

  na = X_test[X_test.isna().any(axis=1)].index
  X_test.drop(na)
  Y_test.drop(na)

  clf.fit(X,Y)
  y_pred = clf.predict(X_test)

  print(confusion_matrix(Y_test, y_pred))
  print(recall_score(Y_test, y_pred))
  print(np.mean(cvs))
  print(cvs)
  return

In [173]:
X = pd.DataFrame()
n_hist = 3

X['XMR-USD Open'] = df_train['XMR-USD Open']
X['XMR-USD High'] = df_train['XMR-USD High']
X['XMR-USD Low'] = df_train['XMR-USD Low']
X['XMR-USD Volume'] = df_train['XMR-USD Volume']

print(X.columns)

cvscore(X, n_hist)

Index(['XMR-USD Open', 'XMR-USD High', 'XMR-USD Low', 'XMR-USD Volume'], dtype='object')
[[ 4  5]
 [ 8 10]]
0.5555555555555556
0.44360217911241034
[0.48826291 0.38028169 0.43661972 0.42253521 0.42253521 0.41314554
 0.43396226 0.41981132 0.5        0.51886792]


In [178]:
X = pd.DataFrame()
n_hist = 3

X['Percentage Change'] = df_train['Percentage Change'].copy()

print(X.columns)

cvscore(X, n_hist)

Index(['Percentage Change'], dtype='object')
[[ 6  3]
 [ 6 12]]
0.6666666666666666
0.5084972096731332
[0.49765258 0.5258216  0.48826291 0.4600939  0.5258216  0.48826291
 0.5754717  0.55188679 0.45283019 0.51886792]


In [179]:
X = pd.DataFrame()
n_hist = 3

X['XMR-USD Open'] = df_train['XMR-USD Open']
X['XMR-USD High'] = df_train['XMR-USD High']
X['XMR-USD Low'] = df_train['XMR-USD Low']
X['XMR-USD Volume'] = df_train['XMR-USD Volume']
X['Percentage Change'] = df_train['Percentage Change']

print(X.columns)

cvscore(X, n_hist)

Index(['XMR-USD Open', 'XMR-USD High', 'XMR-USD Low', 'XMR-USD Volume',
       'Percentage Change'],
      dtype='object')
[[ 5  4]
 [ 5 13]]
0.7222222222222222
0.4694414917176012
[0.51643192 0.40375587 0.43661972 0.46948357 0.50234742 0.45539906
 0.53773585 0.48584906 0.43396226 0.45283019]


In [180]:
X = pd.DataFrame()
n_hist = 5

X['XMR-USD Open'] = df_train['XMR-USD Open']
X['XMR-USD High'] = df_train['XMR-USD High']
X['XMR-USD Low'] = df_train['XMR-USD Low']
X['XMR-USD Volume'] = df_train['XMR-USD Volume']
X['Percentage Change'] = df_train['Percentage Change']

print(X.columns)

cvscore(X, n_hist)

Index(['XMR-USD Open', 'XMR-USD High', 'XMR-USD Low', 'XMR-USD Volume',
       'Percentage Change'],
      dtype='object')
[[ 6  3]
 [ 2 14]]
0.875
0.4708477278766942
[0.4741784  0.48356808 0.45539906 0.38967136 0.47641509 0.4245283
 0.49056604 0.50471698 0.49056604 0.51886792]


In [208]:
X = pd.DataFrame()
n_hist = 1

X['XMR-USD Open'] = df_train['XMR-USD Open']
X['XMR-USD High'] = df_train['XMR-USD High']
X['XMR-USD Low'] = df_train['XMR-USD Low']
X['XMR-USD Volume'] = df_train['XMR-USD Volume']
X['Percentage Change'] = df_train['Percentage Change']

print(X.columns)

cvscore(X, n_hist)

Index(['XMR-USD Open', 'XMR-USD High', 'XMR-USD Low', 'XMR-USD Volume',
       'Percentage Change'],
      dtype='object')
Int64Index([], dtype='int64')
[[ 7  4]
 [ 6 12]]
0.6666666666666666
0.4727234476038622
[0.53051643 0.42723005 0.48356808 0.44600939 0.48356808 0.44131455
 0.49295775 0.52112676 0.40566038 0.49528302]


In [198]:
X = pd.DataFrame()
n_hist = 3

X['BTC-USD Close'] = df_train['BTC-USD Close'].copy()
X['Percentage Change'] = df_train['Percentage Change'].copy()
print(X.columns)
Y = df_train['Target']


''''BTC-USD Close'
'BtcChanePercent'
'EthChangePercent'
'RSI'
'SMA'
'EMA'
'GC=F Close'
'GoldChangePercent'''''
cvscore(X, n_hist)

Index(['BTC-USD Close', 'Percentage Change'], dtype='object')
[[ 2  7]
 [ 2 16]]
0.8888888888888888
0.46665337939587215
[0.5399061  0.45070423 0.46478873 0.3943662  0.46478873 0.38028169
 0.51415094 0.45283019 0.49056604 0.51415094]


In [197]:
X = pd.DataFrame()
n_hist = 3

X['GoldChangePercent'] = df_train['GoldChangePercent'].copy()
X['BTC-USD Close'] = df_train['BTC-USD Close'].copy()
X['Percentage Change'] = df_train['Percentage Change'].copy()
print(X.columns)
Y = df_train['Target']


''''BTC-USD Close'
'BtcChanePercent'
'EthChangePercent'
'RSI'
'SMA'
'EMA'
'GC=F Close'
'GoldChangePercent'''''
cvscore(X, n_hist)

Index(['GoldChangePercent', 'BTC-USD Close', 'Percentage Change'], dtype='object')
[[ 2  7]
 [ 6 12]]
0.6666666666666666
0.47136371689255024
[0.49295775 0.45539906 0.46948357 0.39906103 0.42253521 0.46948357
 0.51886792 0.50943396 0.48113208 0.49528302]


In [202]:
X = pd.DataFrame()
n_hist = 3

X['SMA'] = df_train['SMA'].copy()
X['RSI'] = df_train['RSI'].copy()
X['BTC-USD Close'] = df_train['BTC-USD Close'].copy()
X['Percentage Change'] = df_train['Percentage Change'].copy()
print(X.columns)
Y = df_train['Target']


''''BTC-USD Close'
'BtcChanePercent'
'EthChangePercent'
'RSI'
'SMA'
'EMA'
'GC=F Close'
'GoldChangePercent'''''
cvscore(X, n_hist)

Index(['SMA', 'RSI', 'BTC-USD Close', 'Percentage Change'], dtype='object')
[[ 8  1]
 [13  5]]
0.2777777777777778
0.4369198783868372
[0.46698113 0.4009434  0.36320755 0.41037736 0.45754717 0.38388626
 0.45971564 0.43601896 0.46919431 0.52132701]


In [206]:
X = pd.DataFrame()
n_hist = 3

X['GoldChangePercent'] = df_train['GoldChangePercent'].copy()
X['BtcChanePercent'] = df_train['BtcChanePercent'].copy()
X['BTC-USD Close'] = df_train['BTC-USD Close'].copy()
X['Percentage Change'] = df_train['Percentage Change'].copy()
print(X.columns)
Y = df_train['Target']


''''BTC-USD Close'
'BtcChanePercent'
'EthChangePercent'
'RSI'
'SMA'
'EMA'
'GC=F Close'
'GoldChangePercent'''''
cvscore(X, n_hist)

Index(['GoldChangePercent', 'BtcChanePercent', 'BTC-USD Close',
       'Percentage Change'],
      dtype='object')
Int64Index([], dtype='int64')
[[ 3  6]
 [ 6 12]]
0.6666666666666666
0.4835924351138276
[0.53051643 0.45070423 0.45539906 0.48356808 0.3943662  0.46948357
 0.55188679 0.51886792 0.48584906 0.49528302]


In [207]:
X = pd.DataFrame()
n_hist = 1

X['GoldChangePercent'] = df_train['GoldChangePercent'].copy()
X['BtcChanePercent'] = df_train['BtcChanePercent'].copy()
X['BTC-USD Close'] = df_train['BTC-USD Close'].copy()
X['Percentage Change'] = df_train['Percentage Change'].copy()
print(X.columns)
Y = df_train['Target']


''''BTC-USD Close'
'BtcChanePercent'
'EthChangePercent'
'RSI'
'SMA'
'EMA'
'GC=F Close'
'GoldChangePercent'''''
cvscore(X, n_hist)

Index(['GoldChangePercent', 'BtcChanePercent', 'BTC-USD Close',
       'Percentage Change'],
      dtype='object')
Int64Index([], dtype='int64')
[[ 2  9]
 [ 6 12]]
0.6666666666666666
0.48123173000265745
[0.50234742 0.46948357 0.42723005 0.46948357 0.42253521 0.43661972
 0.5258216  0.53521127 0.51886792 0.50471698]
